<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/RFM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [2]:
# подключение к БД
engine = create_engine(userdata.get('simulative'))

In [3]:
# для удобства напишем функцию для загрузки результатов sql запросов
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [4]:
# запросим предагреггированные данные
sql = """
SELECT
  card as client_id,
  Max((SELECT max(datetime) :: date FROM bonuscheques) - datetime :: date) as days_passed,
  count(*) as cnt,
  sum(summ_with_disc) as summ
FROM
  bonuscheques
GROUP BY
  card
"""
data = select(sql)

In [5]:
# посмотрим, как выглядят данные
data.head()

,client_id,days_passed,cnt,summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577
1,2000200229544,234,2,1657
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111
3,2000200171775,236,2,1033
4,2000200229094,165,2,1235


In [6]:
data.describe()

,days_passed,cnt,summ
count,9394.000000,9394.00000,9394.000000
mean,202.197147,4.09687,3416.820098
std,98.744462,6.43155,6388.046844
min,0.000000,1.00000,21.000000
25%,123.000000,1.00000,731.250000
50%,223.000000,2.00000,1586.000000
75%,290.000000,5.00000,3731.750000
max,332.000000,217.00000,162687.000000


Видим аномальный максимум в признаке cnt (Кто-то покупает почти каждый день?)

In [7]:
data["log_summ"] = data.summ.apply(np.log)

In [8]:
data.head()

,client_id,days_passed,cnt,summ,log_summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577,9.356776
1,2000200229544,234,2,1657,7.412764
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111,9.401869
3,2000200171775,236,2,1033,6.940222
4,2000200229094,165,2,1235,7.118826


In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV

In [10]:
cols_to_test = ["days_passed", "cnt", "log_summ"]
cols_to_test_2 = ["days_passed", "cnt", "summ"]

In [11]:
col_selector = ColumnTransformer(
    transformers=[
        ('selector', "passthrough", cols_to_test)
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform="pandas")      # Трансформер будет возвращать pandas

In [12]:
col_transformer = ColumnTransformer(
    transformers=[
        ('StandardScaler', StandardScaler(), selector(dtype_include="number"))
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas

In [13]:
class DBSCANWrapper(DBSCAN):
    def predict(self,X):
      return self.labels_.astype(int)

In [14]:
pipe_dt_basic = Pipeline(
    [
        ("col_selector", col_selector),
        ("col_transformer", col_transformer),
    ]
)

In [15]:
pipe_dt_dbscan = Pipeline(
    [
        ("col_selector", col_selector),
        ("col_transformer", col_transformer),
        ('simple_model', DBSCANWrapper(n_jobs=1))
    ]
)

In [16]:
dbscan_grid = {
    'simple_model__eps': [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2, 2.5, 3, 4, 5],
    'simple_model__min_samples': [15, 20, 30, 35, 40, 45, 50],
    'simple_model__metric': ['euclidean', 'manhattan'],
    #'col_selector': [ColumnTransformer(transformers=[('selector', 'passthrough', ['days_passed', 'cnt', 'log_summ'])], verbose_feature_names_out=False), ColumnTransformer(transformers=[('selector', 'passthrough', ['days_passed', 'cnt', 'summ'])], verbose_feature_names_out=False)]
}

In [17]:
def score_func(df, labels, n_min=4):
    if len(set(labels)) >= n_min:
      return silhouette_score(df, labels)
    else:
      return -100

In [18]:
custom_metrics = make_scorer(score_func, greater_is_better=True)

In [19]:
search_dbscan = GridSearchCV(pipe_dt_dbscan, dbscan_grid,
                      cv=[(slice(None), slice(None))], scoring=custom_metrics, )

In [ ]:
search_dbscan.fit(data, pipe_dt_basic.fit_transform(data))

GridSearchCV(cv=[(slice(None, None, None), slice(None, None, None))],
             estimator=Pipeline(steps=[('col_selector',
                                        ColumnTransformer(transformers=[('selector',
                                                                         'passthrough',
                                                                         ['days_passed',
                                                                          'cnt',
                                                                          'log_summ'])],
                                                          verbose_feature_names_out=False)),
                                       ('col_transformer',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7916c3bb6050>)],
                                                          verbose_feature_names_out=False)),
                                       ('simple_model',
                                        DBSCANWrapper(n_jobs=1))]),
             param_grid={'simple_model__eps': [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2,
                                               2.5, 3, 4, 5],
                         'simple_model__metric': ['euclidean', 'manhattan'],
                         'simple_model__min_samples': [15, 20, 30, 35, 40, 45,
                                                       50]},
             scoring=make_scorer(score_func))

In [ ]:
print(f"Best parameter (CV score={search_dbscan.best_score_:.5f}):")
print(search_dbscan.best_params_)

Best parameter (CV score=-0.04177):
{'simple_model__eps': 0.1, 'simple_model__metric': 'euclidean', 'simple_model__min_samples': 35}


In [ ]:
labels = search_dbscan.predict(data)
unique, counts = np.unique(labels, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  -1 9151]
 [   0   77]
 [   1  128]
 [   2   38]]


In [21]:
pipe_kMeans = Pipeline(
    [
        ("col_selector", col_selector),
        ("col_transformer", col_transformer),
        ('simple_model', KMeans(random_state=42, n_init=1))
    ]
)

In [40]:
kMeans_grid = {
    'simple_model__n_clusters': list(range(8, 30)),
}

In [41]:
search_kMeans = GridSearchCV(pipe_kMeans, kMeans_grid,
                      cv=[(slice(None), slice(None))], scoring=custom_metrics, verbose=3)

In [42]:
search_kMeans.fit(data, pipe_dt_basic.fit_transform(data))

Fitting 1 folds for each of 22 candidates, totalling 22 fits
[CV 1/1] END ........simple_model__n_clusters=8;, score=0.312 total time=   1.5s
[CV 1/1] END ........simple_model__n_clusters=9;, score=0.309 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=10;, score=0.300 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=11;, score=0.317 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=12;, score=0.308 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=13;, score=0.313 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=14;, score=0.313 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=15;, score=0.302 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=16;, score=0.301 total time=   1.2s
[CV 1/1] END .......simple_model__n_clusters=17;, score=0.301 total time=   1.3s
[CV 1/1] END .......simple_model__n_clusters=18;, score=0.301 total time=   1.6s
[CV 1/1] END .......simple_model__n_clusters=19;

GridSearchCV(cv=[(slice(None, None, None), slice(None, None, None))],
             estimator=Pipeline(steps=[('col_selector',
                                        ColumnTransformer(transformers=[('selector',
                                                                         'passthrough',
                                                                         ['days_passed',
                                                                          'cnt',
                                                                          'log_summ'])],
                                                          verbose_feature_names_out=False)),
                                       ('col_transformer',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7b109a3c36a0>)],
                                                          verbose_feature_names_out=False)),
                                       ('simple_model',
                                        KMeans(n_init=1, random_state=42))]),
             param_grid={'simple_model__n_clusters': [8, 9, 10, 11, 12, 13, 14,
                                                      15, 16, 17, 18, 19, 20,
                                                      21, 22, 23, 24, 25, 26,
                                                      27, 28, 29]},
             scoring=make_scorer(score_func), verbose=3)

In [35]:
search_kMeans.predict(data)

array([3, 0, 1, ..., 0, 2, 0], dtype=int32)

In [43]:

import plotly.express as px

fig = px.scatter_3d(data, x="days_passed" , y="cnt", z="summ", color=search_kMeans.predict(data))
fig.show()

In [36]:
data

,client_id,days_passed,cnt,summ,log_summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577,9.356776
1,2000200229544,234,2,1657,7.412764
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111,9.401869
3,2000200171775,236,2,1033,6.940222
4,2000200229094,165,2,1235,7.118826
...,...,...,...,...,...
9389,4ed88fac-cc11-4be3-9665-e6d46d58eb91,298,4,603,6.401917
9390,258af072-f4f2-49a3-83fb-a93272d43f20,282,1,137,4.919981
9391,2000200195580,300,1,509,6.232448
9392,b6b7fc54-8552-42a2-bfab-51a1ff788bbc,76,1,247,5.509388
